In [ ]:
import time
import json
import jwt
import requests
import getpass
import os
jwks_path="/etc/slurm/jwks.json"
jwks=json.load(open(jwks_path))

def generate_jwt(payload={}, expire=1800):
    current_time_in_seconds = round(time.time())
    expiry_time_in_seconds = current_time_in_seconds + expire
    claims={"exp": expiry_time_in_seconds, "iat":current_time_in_seconds}
    private_key = jwt.algorithms.RSAAlgorithm.from_jwk(json.dumps(jwks["keys"][0]))
    return jwt.encode( claims | payload , private_key, algorithm="RS256")

test the function and decode

In [ ]:
token=generate_jwt({"hello":"world"})
jwt.decode(token,algorithms="RS256",options={"verify_signature": False})

sign a token for slurm rest api

In [ ]:
payload = {
    "homedir": os.path.expanduser("~"),
    "username": getpass.getuser(),
    "uid": os.getuid(),
    "gid": os.getgid()
}
headers = {
    "X-SLURM-USER-NAME": getpass.getuser(),
    "X-SLURM-USER-TOKEN": generate_jwt(payload)
}
# jwt.decode(token,algorithms="RS256",options={"verify_signature": False})

In [ ]:
response = requests.get("http://127.0.0.1/slurm/v0.0.40/diag", headers=headers)
response.json()